In [ ]:
# Importem les funcions del mòdul
from astrosismologia_utils import (
    load_data, process_central_mode, process_amplitude_mode,
    compute_pairwise_differences, compute_autocorrelation,
    plot_spectrum, plot_histogram, plot_spectrum_with_autocorrelation,
    save_results
)
import numpy as np

## 1. Càrrega de dades

In [ ]:
freqs, amps = load_data(DATA_FILE, freq_min=FREQ_MIN)
print(f"Dades carregades: {len(freqs)} punts")
print(f"Rang de freqüències: {freqs.min():.3f} - {freqs.max():.3f} {FREQ_UNIT}")
if FREQ_MIN:
    print(f"Filtrat soroll < {FREQ_MIN} {FREQ_UNIT}")

## 2. Detecció i selecció de pics

In [ ]:
# Processar segons el mode
if SELECTION_MODE == 'amplitude':
    initial_peaks, merged_peaks, selected_peaks, global_max_idx = process_amplitude_mode(
        freqs, amps, num_peaks=NUM_PEAKS,
        freq_range_min=FREQ_RANGE_MIN, freq_range_max=FREQ_RANGE_MAX,
        freq_unit=FREQ_UNIT
    )
else:
    initial_peaks, merged_peaks, selected_peaks, global_max_idx = process_central_mode(
        freqs, amps, num_peaks=NUM_PEAKS,
        threshold_db=THRESHOLD_DB, freq_unit=FREQ_UNIT
    )

## 3. Visualització

In [ ]:
# Visualitzar amb o sense pics fusionats segons el mode
show_merged = SELECTION_MODE == 'central'  # Només mostrar pics fusionats per Sol
plot_spectrum(freqs, amps, initial_peaks, merged_peaks, selected_peaks, global_max_idx, 
              freq_unit=FREQ_UNIT, show_merged=show_merged)

## 4. Anàlisi de diferències

In [6]:
# Calculem diferències
pairwise_diffs = compute_pairwise_differences(freqs, selected_peaks)
deltas = np.array([d[1] for d in pairwise_diffs])

print(f"Diferència mínima: {deltas.min():.6f} mHz")
print(f"Diferència màxima: {deltas.max():.6f} mHz")
print(f"Diferència mitjana: {deltas.mean():.6f} mHz")

## 5. Histograma

In [ ]:
# Calcular autocorrelació només per estrelles (mode amplitude)
autocorr_peak = None
if SELECTION_MODE == 'amplitude':
    lags, autocorr, autocorr_peak = compute_autocorrelation(
        freqs, amps, 
        freq_range_min=FREQ_RANGE_MIN,
        freq_range_max=FREQ_RANGE_MAX,
        exclude_near_zero=0.05
    )
    print(f"\nPic màxim d'autocorrelació: {autocorr_peak:.3f} {FREQ_UNIT}")

# Per al Sol (mode central), no excloure zona propera a zero
exclude_near_zero_hist = (SELECTION_MODE == 'amplitude')
plot_histogram(deltas, bin_width=BIN_WIDTH, freq_unit=FREQ_UNIT, 
               autocorr_peak=autocorr_peak, exclude_near_zero=exclude_near_zero_hist)

## 5b. Visualització espectre i autocorrelació

In [ ]:
# Visualització combinada només per estrelles (mode amplitude)
if SELECTION_MODE == 'amplitude' and autocorr_peak is not None:
    plot_spectrum_with_autocorrelation(
        freqs, amps, selected_peaks,
        lags, autocorr, autocorr_peak,
        freq_unit=FREQ_UNIT,
        title_prefix=DATA_FILE.split('/')[-1].replace('.csv', '').replace('dades/', ''),
        freq_range_min=FREQ_RANGE_MIN,
        freq_range_max=FREQ_RANGE_MAX
    )

## 6. Exportació de resultats

In [ ]:
# Recalculem l'histograma per tenir counts i edges
bins = np.arange(0.0, deltas.max() + BIN_WIDTH, BIN_WIDTH)
counts, edges = np.histogram(deltas, bins=bins)

save_results(freqs, amps, selected_peaks, global_max_idx, 
             pairwise_diffs, counts, edges, OUTPUT_DIR, BIN_WIDTH, FREQ_UNIT)